In [2]:
import pathlib
import os
import cv2
from airo_camera_toolkit.utils import ImageConverter
from microplate.opencv_annotation import get_manual_annotations_image , Annotation
import numpy as np
from microplate import PACKAGE_PATH
from microplate.camera_pose import get_camera_pose, pose_model_to_homogeneous_matrix

In [3]:

images_to_label_indices = [0,10,20,-1]
keypoint_names = ["topleft"]#,"topright","bottomleft"]
annotation_config = {keypoint: Annotation.Keypoint for keypoint in keypoint_names}

directory = PACKAGE_PATH.parents[1] / "scene_images"
images = os.listdir(directory)
images = [image for image in images if image.endswith(".png")]
images = sorted(images)
keypoint_dict = {image: {} for image in images}
manually_labeled_keypoints = {images[id]: {} for id in images_to_label_indices}
for image_path, _ in manually_labeled_keypoints.items():
    image = cv2.imread(str(directory / image_path))
    image = ImageConverter.from_opencv_format(image).image_in_numpy_format
    annotation = get_manual_annotations_image(image, annotation_config)
    annotation = {key: np.array(value) for key, value in annotation.items()}
    manually_labeled_keypoints[image_path] = annotation


qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 219, resource id: 0, major code: 140 (Unknown), minor code: 20


In [4]:

keypoint_names

['topleft']

In [ ]:
import json
from airo_dataset_tools.data_parsers.pose import Pose
from microplate.multi_view_triangulation import multiview_triangulation_midpoint

for keypoint in keypoint_names:
    print(keypoint)
    labeled_extrinsics = []
    labeled_intrinsics = []
    labeled_image_points = []
    for image_path, image_points in manually_labeled_keypoints.items():
        extrinsics_path = image_path.replace(".png","_camera_pose.json")
        extrinsics = Pose(**json.load(open(directory/extrinsics_path)))
        extrinsics = pose_model_to_homogeneous_matrix(extrinsics)
        intrinsics_path = image_path.replace(".png","_camera_intrinsics.npy")
        camera_intrinsics = np.load(str(directory/intrinsics_path))
        image_point = np.array(image_points[keypoint])
        labeled_extrinsics.append(extrinsics)
        labeled_intrinsics.append(camera_intrinsics)
        labeled_image_points.append(image_point)

    # triangulate the points
    print(labeled_image_points)
    keypoint_position = multiview_triangulation_midpoint(labeled_extrinsics,labeled_intrinsics,labeled_image_points)
    print(f"{keypoint} position: {keypoint_position}")

    for image_path in keypoint_dict.keys():
        extrinsics_path = image_path.replace(".png","_camera_pose.json")
        extrinsics = Pose(**json.load(open(directory/extrinsics_path)))
        extrinsics = pose_model_to_homogeneous_matrix(extrinsics)
        intrinsics_path = image_path.replace(".png","_camera_intrinsics.npy")
        camera_intrinsics = np.load(str(directory/intrinsics_path))
        image_point = camera_intrinsics @ (np.linalg.inv(extrinsics) @ np.append(keypoint_position,1))[:3]
        image_point = image_point[:2] / image_point[2]
        keypoint_dict[image_path][keypoint] = image_point
        image = cv2.imread(str(directory/image_path))
        cv2.circle(image,(int(image_point[0]),int(image_point[1])),5,(0,0,255),-1)
        cv2.imshow("image",image)
        cv2.waitKey(0)

json.dump(keypoint_dict,open(directory/"keypoints.json","w"),indent=4)

In [6]:
keypoint_dict

{'20230803_165759.png': {'topleft': array([799.19566852, 491.03004186])},
 '20230803_165802.png': {'topleft': array([725.03412684, 535.84790661])},
 '20230803_165804.png': {'topleft': array([841.63872907, 507.63955699])},
 '20230803_165808.png': {'topleft': array([722.54288233, 592.97837736])},
 '20230803_165813.png': {'topleft': array([902.06190256, 452.04679299])},
 '20230803_165818.png': {'topleft': array([685.72430634, 644.59945159])},
 '20230803_165829.png': {'topleft': array([853.80485223, 196.1732616 ])},
 '20230803_165840.png': {'topleft': array([372.86653206, 260.64509735])},
 '20230803_165847.png': {'topleft': array([682.28450146, 123.24649365])},
 '20230803_165852.png': {'topleft': array([499.92710369, 146.39781549])},
 '20230803_165855.png': {'topleft': array([625.66721607, 157.65268251])},
 '20230803_165858.png': {'topleft': array([532.11426546, 172.098753  ])},
 '20230803_165900.png': {'topleft': array([607.87248831, 197.01934148])},
 '20230803_165902.png': {'topleft': ar

: 